# 🦌 ELK Transformer 🤖

A transformer object that will convert some input source into valid Elk Json. This
example using a transformer around a networkx graph.

In [2]:
import json
import pathlib

import ipywidgets

import networkx
import traitlets
from IPython.display import display

import ipyelk
import ipyelk.nx
# import ipyelk.tools



## Flat structure

A `networkx.MultiDigraph` can be used to create a flat graph.

> _TODO: There should be an option to specify if ports should be created or only connect
> edges between the nodes_

In [3]:
def a_flat_elk_json_example(graph: networkx.MultiDiGraph = None):
    graph = graph or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("flat_graph.json").read_text(encoding="utf-8"))
    )
    elk = ipyelk.from_nx(
        graph=graph,
        layout=dict(min_height="200px")
    )
#     elk = ipyelk.nx.Diagram(
#         source=ipyelk.nx.NXSource(
#             graph = graph,
#         ),
#         layout=dict(min_height="200px")
#     )
    return elk

In [4]:
if __name__ == "__main__":
    flat = a_flat_elk_json_example()
    display(flat)

Diagram(children=[HTML(value='<style></style>', layout=Layout(display='None')), SprottyViewer(hover=Hover(tee=…




## Hierarchical Diagram with Ports

In [7]:
def a_hierarchical_elk_example(
    tree: networkx.MultiDiGraph = None, ports: networkx.MultiDiGraph = None
):
    tree = tree or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("hier_tree.json").read_text(encoding="utf-8"))
    )
    ports = ports or networkx.readwrite.json_graph.node_link_graph(
        json.loads(pathlib.Path("hier_ports.json").read_text(encoding="utf-8"))
    )
    elk = ipyelk.from_nx(
        graph=ports,
        hierarchy = tree,
        layout=dict(min_height="200px")
    )
        
#     elk = ipyelk.nx.Diagram(
#         source=ipyelk.nx.NXSource(
#             graph = ports,
#             hierarchy = tree
#         ),
#         layout=dict(min_height="200px")
#     )
    return elk

elk = a_hierarchical_elk_example()

elk

Diagram(children=[HTML(value='<style></style>', layout=Layout(display='None')), SprottyViewer(hover=Hover(tee=…

In [28]:
from ipyelk.pipes.visibility import exclude_hidden, VisibilityPipe
from ipyelk.elements.serialization import from_elk_json
from ipyelk.elements import VisIndex, ElementIndex
from IPython.display import JSON


# root = elk.view.source.value
root.children[0].children[0].properties.hidden=True
vis_index = VisIndex.from_els(root)
len(vis_index)

11

In [17]:
for p in elk.pipe.pipes:
    if isinstance(p, VisibilityPipe):
        break
p

VisibilityPipe()

In [30]:
import asyncio
root.children[0].children[0].properties.hidden=False
task = asyncio.create_task(elk.pipe.run(start_pipe=p, value=root))

In [10]:
root.edges[0]

Edge(id='27932a90-4eb6-4d66-9c4c-ee9742fc81ac', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=EdgeProperties(cssClasses='', shape=None, key=None, hidden=None), source=Port(id='6efae1db-b6f8-4330-bca3-a8d9c1e61871.ba9a19af-d306-4860-aa72-54e9709fecca', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=PortProperties(cssClasses='', shape=None, key='y', hidden=None), x=-5.0, y=55.015625, width=5.0, height=5.0), target=Port(id='cbcc9a05-e31f-40e2-ae39-74dd77b10440.034a3d75-24ef-4014-b29b-182846c0a77e', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=PortProperties(cssClasses='', shape=None, key='y', hidden=None), x=-5.0, y=40.0, width=5.0, height=5.0), sections=[EdgeSection(id='27932a90-4eb6-4d66-9c4c-ee9742fc81ac_s0', startPoint=Point(x=34.0, y=110.03125), endPoint=Point(x=147.96875, y=72.515625), bendPoints=[Point(x=24.0, y=110.03125), Point(x=24.0, y=144.53125), Point(x=137.96875, y=144.53125), Point(x=137.96875, y=72.515625)],

In [17]:

index2 = ElementIndex.from_els(p.outlet.value)
# index2.get('d92a873d-1d35-47ab-b746-cde160c51933.feefdeb2-117e-4c9f-8e33-4ceb0e93df80')

KeyError: 'd92a873d-1d35-47ab-b746-cde160c51933.feefdeb2-117e-4c9f-8e33-4ceb0e93df80'

In [10]:
port = p.outlet.value.children[0].ports[0]
port.id, port.properties.key

('56ab23e9-5046-4b2d-b8a5-319375483c39.c22fa774-d367-42dd-ba0e-cba0424d39d3',
 '56ab23e9-5046-4b2d-b8a5-319375483c39.c22fa774-d367-42dd-ba0e-cba0424d39d3')

In [11]:
JSON(p.outlet.value.dict())

<IPython.core.display.JSON object>

In [ ]:
1+1

In [13]:
%debug

> /home/dfreeman6/Documents/ipyelk/examples/pydantic/main.py(400)pydantic.main.BaseModel.__init__()



ipdb>  u


> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/elements/index.py(71)from_els()
     69         return cls(
     70             elements = elements,
---> 71             vis_index = vis_index,
     72         )
     73 



ipdb>  u


> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/elements/serialization.py(38)from_elkjson()
     36     edges_map = pop_edges(data)  # dict of node.id to edge list
     37     root = Node(**data)  # new element hierarchy without edges
---> 38     el_map = ElementIndex.from_els(root, vis_index=vis_index)  # get mapping of ids to elements
     39     el_map.link_edges(edges_map)
     40     # reapplies edges to `data`



...

ipdb>  u


> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/elements/serialization.py(61)from_elk_json()
     59     if js is None:
     60         return None
---> 61     return from_elkjson(js)
     62 
     63 



ipdb>  js


{}


ipdb>  c


In [6]:
JSON(root.dict())


<IPython.core.display.JSON object>

In [7]:
with exclude_hidden:
    data = root.dict()
#     elk.view.source.value = from_elk_json(data, None)
id(elk.view.source.value)

140538878876720

In [8]:
JSON(data)

<IPython.core.display.JSON object>

In [9]:
from ipyelk.elements.serialization import pop_edges, ElementIndex, apply_edges
from ipyelk.elements import Node, Registry, index, Port

context = Registry()

ports = []
with context:

    edges_map = pop_edges(data)  # dict of node.id to edge list
    root = Node(**data)  # element hierarchy without edges
    el_map = ElementIndex.from_els(root, vis_index=vis_index)  # get dict of id to element
    el_map.link_edges(edges_map)
    # reapplies edges to `data`

    apply_edges(data, edges_map)
    
    for el in index.iter_elements(root):
        
        el.id = el.get_id()
        if isinstance(el, Port):
            print(el.id)
            ports.append(el)
el


652b428c-82c4-44bb-823b-cdef69b610e3.a348080a-6e38-44e6-9af1-21b85f55729b
652b428c-82c4-44bb-823b-cdef69b610e3.e29d567a-31d8-4692-b864-89a32ca36331
652b428c-82c4-44bb-823b-cdef69b610e3.6a37252a-983f-477e-82c4-a55c8dac90a2
1e5f0691-de00-4e38-af6b-923d8bf6e5e3.d7c980bb-7287-4b18-9b46-7b561a4c97b9
1e5f0691-de00-4e38-af6b-923d8bf6e5e3.ec735dfd-37fa-4807-8daf-30cdfb914e3b
1e5f0691-de00-4e38-af6b-923d8bf6e5e3.032242fe-f68d-4315-99d4-937df18fdeab


Edge(id='73e1d989-7cd8-4c4b-81de-0c6cd0c672e0', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=EdgeProperties(cssClasses='slack-edge', shape=None, key=None, hidden=None), source=Port(id='652b428c-82c4-44bb-823b-cdef69b610e3.6a37252a-983f-477e-82c4-a55c8dac90a2', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=PortProperties(cssClasses='slack-port', shape=None, key='3ab34ba7-0ac9-4d5f-aca2-d674b87b6247.3264be9d-6834-4ada-bbc4-7c382669527a', hidden=None), x=None, y=None, width=None, height=None), target=Port(id='1e5f0691-de00-4e38-af6b-923d8bf6e5e3.032242fe-f68d-4315-99d4-937df18fdeab', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=PortProperties(cssClasses='', shape=None, key=None, hidden=None), x=-5.0, y=25.0, width=5.0, height=5.0), sections=[EdgeSection(id='73e1d989-7cd8-4c4b-81de-0c6cd0c672e0_s0', startPoint=Point(x=83.96875, y=102.53125), endPoint=Point(x=147.96875, y=57.515625), bendPoints=[Point(x=127.96875, y=102.531

In [10]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [11]:
for el in index.iter_elements(root):
    print(type(el), el.id)

<class 'ipyelk.elements.elements.Node'> 21d47673-3daa-4e38-846d-d0a596fb4374
<class 'ipyelk.elements.elements.Node'> n0
<class 'ipyelk.elements.elements.Port'> 20d2c4d7-281c-47ed-af38-c52d4c8586e9.91a87627-805a-44a0-b7da-d3793598c81e
<class 'ipyelk.elements.elements.Port'> 20d2c4d7-281c-47ed-af38-c52d4c8586e9.26710b11-a010-444a-8e80-f57666f51594
<class 'ipyelk.elements.elements.Port'> 20d2c4d7-281c-47ed-af38-c52d4c8586e9.1a0c8c43-c25a-4aa4-a88d-cbfc76114fe7
<class 'ipyelk.elements.elements.Label'> be3b8140-979e-4f09-84fe-da55862dfc60
<class 'ipyelk.elements.elements.Node'> n3
<class 'ipyelk.elements.elements.Port'> 352af3a5-70f5-42fa-b9c7-9bcd05147905.a3665f29-6cfe-4134-a5b0-fd38c8828ea9
<class 'ipyelk.elements.elements.Port'> 352af3a5-70f5-42fa-b9c7-9bcd05147905.3420261a-776c-4bef-a0b5-aa27f5d3583c
<class 'ipyelk.elements.elements.Port'> 352af3a5-70f5-42fa-b9c7-9bcd05147905.0b4d81ca-29a0-4ac6-b71c-a560c146bb45
<class 'ipyelk.elements.elements.Label'> 885c0293-9409-45b8-b1d8-cea2ed753a

In [12]:
e = root.edges[0]
with context:
    print(e.get_id(), e.source.get_id(), e.source.properties.key)

9dbe8941-da06-4da0-ae53-eaa719ea5c8b 20d2c4d7-281c-47ed-af38-c52d4c8586e9.91a87627-805a-44a0-b7da-d3793598c81e 07f4444d-879f-434f-996d-b7a53336be40.3602cd2f-e7f7-43fb-8eb4-092e8954e751


In [13]:
root.dict()

{'id': '21d47673-3daa-4e38-846d-d0a596fb4374',
 'labels': [],
 'layoutOptions': {'org.eclipse.elk.hierarchyHandling': 'INCLUDE_CHILDREN'},
 'properties': {'cssClasses': '', 'shape': None, 'key': None, 'hidden': None},
 'x': 0.0,
 'y': 0.0,
 'width': 187.9375,
 'height': 169.03125,
 'ports': [],
 'children': [{'id': 'n0',
   'labels': [{'id': 'be3b8140-979e-4f09-84fe-da55862dfc60',
     'layoutOptions': {'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_TOP INSIDE'},
     'properties': {'cssClasses': '',
      'shape': None,
      'key': None,
      'hidden': None,
      'selectable': False},
     'x': 43.7421875,
     'y': 5.0,
     'width': 13.484375,
     'height': 13.015625,
     'text': 'n0',
     'labels': []}],
   'layoutOptions': {'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE'},
   'properties': {'cssClasses': '',
    'shape': None,
    'key': None,
    'hidden': None},
   'x': 12.0,
   'y': 12.0,
   'width': 100.96875,
   'height': 145.03

In [14]:
with context:
    from_elk_json(root.dict(), None)

In [15]:
%debug

ERROR:root:No traceback has been produced, nothing to debug.


In [16]:
root.edges[0].source

Port(id='20d2c4d7-281c-47ed-af38-c52d4c8586e9.91a87627-805a-44a0-b7da-d3793598c81e', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=PortProperties(cssClasses='slack-port', shape=None, key='07f4444d-879f-434f-996d-b7a53336be40.3602cd2f-e7f7-43fb-8eb4-092e8954e751', hidden=None), x=None, y=None, width=None, height=None)

In [17]:
elk.view.source.value=root

In [ ]:
def link_edges(edges_map, el_map):
    for node_id, edges in edges_map.items():
        node = el_map.get(node_id)
        node.edges = [build_edge(e, el_map) for e in edges]

In [11]:
hidden_map['b9323f1a-b397-47ee-9cb3-0fa5876754ae.bd108fcc-023b-4feb-a216-d99b76087e3c']

Node(id='n0', labels=[Label(id='0ca1e840-7c62-4cef-83a8-89f3dee0011a', labels=[], layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_TOP INSIDE'}, metadata=ElementMetadata(), properties=LabelProperties(cssClasses='', shape=None, key=None, hidden=None, selectable=False), x=43.7421875, y=5.0, width=13.484375, height=13.015625, text='n0')], layoutOptions={'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE'}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=None, key=None, hidden=None), x=12.0, y=12.0, width=100.96875, height=145.03125, ports=[], children=[Node(id='n1', labels=[Label(id='47bca8f7-78f8-43c5-bae9-cf17ce539388', labels=[], layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_TOP INSIDE'}, metadata=ElementMetadata(), properties=LabelProperties(cssClasses='', shape=None, key=None, hidden=None, selectable=False), x=22.0, y=5.0, width=12.96875, height=13.015625, text='n1')], layoutOptions={'o

In [10]:
el_map

{'4451a30d-8d9b-40ab-9952-90bb6ef6fee9': Node(id='4451a30d-8d9b-40ab-9952-90bb6ef6fee9', labels=[], layoutOptions={'org.eclipse.elk.hierarchyHandling': 'INCLUDE_CHILDREN'}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=None, key=None, hidden=None), x=0.0, y=0.0, width=187.9375, height=169.03125, ports=[], children=[Node(id='n0', labels=[Label(id='0ca1e840-7c62-4cef-83a8-89f3dee0011a', labels=[], layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_TOP INSIDE'}, metadata=ElementMetadata(), properties=LabelProperties(cssClasses='', shape=None, key=None, hidden=None, selectable=False), x=43.7421875, y=5.0, width=13.484375, height=13.015625, text='n0')], layoutOptions={'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE'}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=None, key=None, hidden=None), x=12.0, y=12.0, width=100.96875, height=145.03125, ports=[], children=[], edges=[]), Node(i

{Node(id='n1', labels=[Label(id='4e30fda2-ecc7-46af-bff0-f70e9b850707', labels=[], layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_TOP INSIDE'}, metadata=ElementMetadata(), properties=LabelProperties(cssClasses='', shape=None, key=None, hidden=None, selectable=False), x=22.0, y=5.0, width=12.96875, height=13.015625, text='n1')], layoutOptions={'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE'}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=None, key=None, hidden=True), x=27.0, y=40.515625, width=56.96875, height=82.015625, ports=[Port(id='82d53903-6fba-4640-a2ae-ad2671aa8e2d.823394af-61f7-4ea9-94ec-23a3fe67c720', labels=[], layoutOptions={}, metadata=ElementMetadata(), properties=PortProperties(cssClasses='', shape=None, key=None, hidden=None), x=-5.0, y=40.015625, width=5.0, height=5.0), Port(id='82d53903-6fba-4640-a2ae-ad2671aa8e2d.e6b6455d-d066-413e-93ba-84d575bc497c', labels=[], layoutOptions={}, met

In [11]:
%debug

> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/elements/serialization.py(21)apply_edges()
     19 
     20 def apply_edges(data, edges):
---> 21     node_id = data["id"]
     22     if node_id in edges:
     23         data["edges"] = edges.get(node_id)



ipdb>  data


{}


ipdb>  u


> /home/dfreeman6/Documents/ipyelk/py_src/ipyelk/elements/serialization.py(25)apply_edges()
     23         data["edges"] = edges.get(node_id)
     24     for child in data.get("children", []):
---> 25         apply_edges(child, edges)
     26     return edges
     27 



ipdb>  child


{}


ipdb>  c


In [7]:
%debug

> /home/dfreeman6/Documents/ipyelk/examples/pydantic/main.py(767)pydantic.main.BaseModel._get_value()



ipdb>  ll


*** could not get source code


ipdb>  u


> /home/dfreeman6/Documents/ipyelk/examples/pydantic/main.py(795)genexpr()



...

ipdb>  self


Node(id='n0', labels=[Label(id='2e9b4876-1b5b-476d-bae7-2fe89bd1cb29', labels=[], layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_TOP INSIDE'}, metadata=ElementMetadata(), properties=LabelProperties(cssClasses='', shape=None, key=None, hidden=None, selectable=False), x=43.7421875, y=5.0, width=13.484375, height=13.015625, text='n0')], layoutOptions={'org.eclipse.elk.nodeSize.constraints': 'NODE_LABELS PORTS PORT_LABELS MINIMUM_SIZE'}, metadata=ElementMetadata(), properties=NodeProperties(cssClasses='', shape=None, key=None, hidden=None), x=12.0, y=12.0, width=100.96875, height=145.03125, ports=[], children=[Node(id='n1', labels=[Label(id='753a02b7-cc30-428a-8c9b-b1a7fd4ec751', labels=[], layoutOptions={'org.eclipse.elk.nodeLabels.placement': 'H_CENTER V_TOP INSIDE'}, metadata=ElementMetadata(), properties=LabelProperties(cssClasses='', shape=None, key=None, hidden=None, selectable=False), x=22.0, y=5.0, width=12.96875, height=13.015625, text='n1')], layoutOptions={'o

ipdb>  ll


     95     def dict(self, **kwargs) -> Dict:
     96         """Shimming in the ability to have excluded fields by default. This
     97         should be removeable in future versions of pydantic
     98         """
     99         excluded = getattr(self.Config, "excluded", [])
    100         if excluded:
    101             kwargs = add_excluded_fields(kwargs, excluded)
--> 102         data = super().dict(**kwargs)
    103         data["id"] = self.get_id()
    104 
    105         # mechanism to convert some fields to a list representation if needed
    106         for key in getattr(self.Config, "to_list", []):
    107             if key in data:
    108                 value = data[key]
    109                 if isinstance(value, (set, list, tuple)):
    110                     value = list(value)
    111                 elif isinstance(value, dict):
    112                     value = list(data[key].values())
    113                 else:
    114                     raise Typ

ipdb>  c


In [8]:
r2 = elk.view.source.value 
print(id(r2))
visible_els, hidden_els = check_visible(r2)
len(visible_els), len(hidden_els)

140189843036912


(11, 11)

In [5]:
from ipyelk.pipes import EMPTY_SENTINEL
from ipyelk.elements import BaseElement, Node, Edge
from typing import Iterator, Tuple

import networkx as nx
def iter_visible(*els: BaseElement, hidden=False) -> Iterator[Tuple[BaseElement, bool]]:
    """Iterate over BaseElements hierarchy and track hidden

    :param el: current element
    :param hidden: containing element is hidden
    :yield: sub element and hidden state
    """
    for el in els:
        hidden = bool(hidden or el.properties.hidden)
        yield el, hidden
        if isinstance(el, Node):
            yield from iter_visible(*el.children, hidden=hidden)
            yield from iter_visible(*el.ports, hidden=hidden)
            yield from iter_visible(*el.edges, hidden=hidden)
        yield from iter_visible(*el.labels, hidden=hidden)


def check_visible(root:BaseElement):
    show = set()
    hide = set()
    for el, hidden in iter_visible(root):
        if hidden:
            hide.add(el)
        else:
            show.add(el)
    return show, hide




In [8]:
# check hidden edges
hidden_edges = [el for el in hidden_els if isinstance(el, Edge)]


In [9]:
root.properties.hidden = None

In [22]:
hidden

In [19]:
len(hierarchy)

22

In [6]:
def a_collapsible_elk_example(elk=None):
    elk = elk or a_hierarchical_elk_example()
#     ports, tree = xelk.source

    toggle = ipywidgets.Button(description="Toggle Collapsed")

    @toggle.on_click
    def toggle_node(widget):
        for element_id in elk.selected:
            if element_id in tree:
                for child in tree.neighbors(element_id):
                    state = tree.nodes[child].get("hidden", False)
                    tree.nodes[child]["hidden"] = not state
                xelk.refresh()

    box = ipywidgets.VBox(
        [
            ipywidgets.HBox(
                [ipywidgets.HTML("<h2>👇 click a group node then click 👉</h2>"), toggle]
            ),
            elk,
        ]
    )
    return box, elk

In [9]:
if __name__ == "__main__":
    hier_box, hier_elk, hier_xelk = a_collapsible_elk_example()
    display(hier_box)

TypeError: cannot unpack non-iterable Diagram object

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).